In [ ]:
object_name = "OB0001"

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import astroalign

In [ ]:
from astropy.nddata import CCDData
from astropy.stats import mad_std

from ccdproc import ImageFileCollection
import ccdproc


In [ ]:
from convenience_functions import show_image, show_images, show_image_residual

In [ ]:
import astroalign

In [ ]:
from pathlib import Path

In [ ]:
import arya
plt.rcParams["figure.dpi"] = 300

In [ ]:
import os

## Setup

In [ ]:
filters = ["Sloan_g", "Sloan_r", "Sloan_i"]

In [ ]:
DATADIR = Path(f"{object_name}")

In [ ]:
for filt in ["g", "r", "i"]:
    for cls in ["obj", "std"]:
        filename = DATADIR / f"reduced/stacked_{cls}_{filt}.fits"
        if filename.is_file():
            os.remove(filename)

In [ ]:
def get_paths(dirname):
    data_dir = DATADIR
    std_dir = data_dir.joinpath(dirname, "stds")
    obj_dir = data_dir.joinpath(dirname, "object")

    return std_dir, obj_dir

In [ ]:
def get_image_files(dirname):
    paths = get_paths(dirname)
    return [ImageFileCollection(path) for path in paths]

In [ ]:
imgs_std, imgs_obj = get_image_files("reduced")

In [ ]:
def split_into_filters(imgs):
    allimgs = {}
    
    for filt in filters:
        ccds = imgs.data(FILTER2=filt)
        allimgs[filt] = [newbyteorder(x) for x in ccds]

    return allimgs

In [ ]:
def split_filenames_into_filters(imgs):
    allimgs = {}
    
    for filt in filters:
        ccds = imgs.files_filtered(FILTER2=filt)
        allimgs[filt] = [(x) for x in ccds]

    return allimgs

In [ ]:
def newbyteorder(x):
    return x.byteswap().view(x.dtype.newbyteorder())

In [ ]:
files_obj = split_filenames_into_filters(imgs_obj)
files_std = split_filenames_into_filters(imgs_std)

In [ ]:
imgs_obj_byfilt = split_into_filters(imgs_obj)
imgs_std_byfilt = split_into_filters(imgs_std)

In [ ]:
def align_all(images):
    image_0 = images[0]
    mask_combined = np.full_like(image_0, False, dtype=bool)

    images_aligned = [image_0]
    for i in range(1, len(images)):
        print(f"aligning image {i} / {len(images)-1}")
        image_aligned, mask = astroalign.register(images[i], image_0)
        mask_combined = mask_combined | mask
        images_aligned.append(image_aligned)

    return images_aligned, mask_combined

In [ ]:
imgs_mask_obj_aligned = {filt: align_all(imgs) for filt, imgs in imgs_obj_byfilt.items()}
imgs_mask_std_aligned = {filt: align_all(imgs) for filt, imgs in imgs_std_byfilt.items()}

imgs_obj_aligned = {filt: imgs for filt, (imgs, mask) in imgs_mask_obj_aligned.items()}
mask_obj_aligned = {filt: mask for filt, (imgs, mask) in imgs_mask_obj_aligned.items()}
imgs_std_aligned = {filt: imgs for filt, (imgs, mask) in imgs_mask_std_aligned.items()}
mask_std_aligned = {filt: mask for filt, (imgs, mask) in imgs_mask_std_aligned.items()}

The above plot shows an example of the alignment process

In [ ]:
def combine_images(filenames, **kwargs):
    return ccdproc.combine(filenames, 
                           method="median",
                           sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5, 
                           sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std, 
                           mem_limit=1e9,
                           **kwargs
                            )

In [ ]:
imgs_obj_stacked = {filt: combine_images([CCDData(img, unit="adu") for img in imgs]) for filt, imgs in imgs_obj_aligned.items()}
imgs_std_stacked = {filt: combine_images([CCDData(img, unit="adu") for img in imgs]) for filt, imgs in imgs_std_aligned.items()}

In [ ]:
imgs_obj_stacked["Sloan_g"].mask = mask_obj_aligned["Sloan_g"]

In [ ]:
for filt in filters:
    imgs_obj_stacked[filt].mask = mask_obj_aligned[filt]
    imgs_std_stacked[filt].mask = mask_std_aligned[filt]

In [ ]:
for filt in ["g", "r", "i"]:
    imgs_obj_stacked[f"Sloan_{filt}"].write(DATADIR / f"reduced/stacked_obj_{filt}.fits")
    imgs_std_stacked[f"Sloan_{filt}"].write(DATADIR / f"reduced/stacked_std_{filt}.fits")

# Plots

In [ ]:
show_image_residual(imgs_obj_aligned["Sloan_r"][1], imgs_obj_byfilt["Sloan_r"][1])

The above image is an example of an alignment shift on an image

In [ ]:
for filt in filters:
    show_image(imgs_obj_stacked[filt])

In [ ]:
for filt in filters:
    show_image(imgs_std_stacked[filt])

## Save residual images

In [ ]:
DATADIR

In [ ]:
(DATADIR / "residual").mkdir(exist_ok=True)
(DATADIR / "residual/object").mkdir(exist_ok=True)
(DATADIR / "residual/stds").mkdir(exist_ok=True)

In [ ]:
for filt in filters:
    for img, name in zip(imgs_obj_aligned[filt], files_obj[filt]):
        img_processed = ccdproc.subtract_bias(CCDData(img, unit="adu"), imgs_obj_stacked[filt])
        filename = DATADIR / "residual/object" / name
        img_processed.write(filename)

In [ ]:
for filt in filters:
    for img, name in zip(imgs_std_aligned[filt], files_std[filt]):
        img_processed = ccdproc.subtract_bias(CCDData(img, unit="adu"), imgs_std_stacked[filt])
        filename = DATADIR / "residual/stds" / name
        img_processed.write(filename)

In [ ]:
for filt in filters:
    for img, name in zip(imgs_obj_aligned[filt], files_obj[filt]):
        show_image_residual(img, imgs_obj_stacked[filt].data, clim=(-1000, 1000))
        img_processed = ccdproc.subtract_bias(CCDData(img, unit="adu"), imgs_obj_stacked[filt])
        plt.title(filt + " " + name)

In [ ]:
for filt in filters:
    for img, name in zip(imgs_std_aligned[filt], files_std[filt]):
        show_image_residual(img, imgs_std_stacked[filt].data)
        plt.title(filt + " " + name)